sheet flow:

- ~~define track of interest based on sheet_gid dict~~
- ~~read in weekend dataframe~~
- ~~run function drop_empties~~
- ~~run function check_df~~
- ~~produce these lists:~~
    - ~~predicted_qualifying_order, actual_qualifying_order, predicted_race_order, actual_race_order~~
- ~~produce these dfs:~~
    - ~~constructor_pricing, driver_pricing~~
- ~~produce these lists:~~
    - ~~drivers, constructors~~
- ~~set up constants for tracking weekend scores~~
    - ~~constructor_scores, driver_scores~~
- ~~run function score_qualification_order~~
- run function score_race_order
- run function score_fastest_pitstop
- run function score_fastest_lap
    
still to-do:
- ~~changing the overtake bonus/gain/loss points from estimations to direct reports in fantasy~~
- ~~produce df of driver and constructor pricing for making team affordability calculations each week, later when i get to the full program part of it~~
- ~~incorporate the team recommendation function that the post i found used~~
- ~~write function scoring the predictions only~~
    - ~~assume difference between predicted qualifying position and predicted race position are direct gain/loss~~
- ~~incorporate the pieces into scoring all of the race into one function~~
    - ~~qualifying, race, superlatives, overtakes+gain/loss~~
- make google sheet of teams and track team composition through each weekend
    - read this in, figure out if i can define a range of cells so i can do all three teams on the same google sheet or if i'm going to have to do a separate sheet for each
- ~~add scoring for my team to use for the team recommendations~~
- ~~add ability to handle DNF, DNQ, and OUT (sainz, stroll, zhou at saudi arabia)~~

In [1]:
from weekend_functions import *

### Define Track and Read In Weekend Data

In [2]:
track_name = 'australia'

In [3]:
# read in from google sheets the sheet for this weekend's events
weekend_df = pd.read_csv(f'https://docs.google.com/spreadsheets/d/14kBO9LAo4-uPrQlH6xm_Fm2OcNB15xUdnzUbIaRFjOU/export?format=csv&gid={sheet_gid[track_name]}')

# read in from local sometimes needed when offline, make sure to download the file from sheets first...
# weekend_df = pd.read_csv(f'f1_main - {track_name}.csv')

weekend_df = drop_empties(weekend_df)
check_df(weekend_df)

weekend_df

,Team,Driver,predicted_qualifying_australia,predicted_race_australia
0,Red Bull Racing-RBPT,Max Verstappen,1,1
1,Red Bull Racing-RBPT,Sergio Perez,4,2
2,Ferrari,Charles Leclerc,2,3
3,Ferrari,Carlos Sainz Jr.,3,4
4,Aston Martin Aramco-Mercedes,Fernando Alonso,8,5
5,Mercedes,George Russell,5,6
6,Mercedes,Lewis Hamilton,7,7
7,McLaren-Mercedes,Lando Norris,6,8
8,McLaren-Mercedes,Oscar Piastri,9,9
9,Aston Martin Aramco-Mercedes,Lance Stroll,10,10


In [4]:
weekend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Team                            20 non-null     object
 1   Driver                          20 non-null     object
 2   predicted_qualifying_australia  20 non-null     int64 
 3   predicted_race_australia        20 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 768.0+ bytes


### Define Predicted Qualifying, Actual Qualifying, Predicted Race, Actual Race Orders, Driver Gain/Loss

this was used when writing functions to score the actual race. its not needed for what this sheet has become, picking a team for the next race. i'll move it to another sheet eventually when if/when i need it still. or i'll delete it.

In [5]:
# # qualifying orders
# predicted_qualifying_order = weekend_df[['Driver', f'predicted_qualifying_{track_name}']].sort_values(by=[f'predicted_qualifying_{track_name}'])['Driver'].tolist()
# actual_qualifying_order = weekend_df[['Driver', f'actual_qualifying_{track_name}']].sort_values(by=[f'actual_qualifying_{track_name}'])['Driver'].tolist()

# # race orders
# predicted_race_order = weekend_df[['Driver', f'predicted_race_{track_name}']].sort_values(by=[f'predicted_race_{track_name}'])['Driver'].tolist()
# if f'actual_race_{track_name}' in weekend_df.columns:
#     actual_race_order = weekend_df[['Driver', f'actual_race_{track_name}']].sort_values(by=[f'actual_race_{track_name}'])['Driver'].tolist()

# # driver_gain_loss
# driver_gain_loss_overtake = {}
# for driver in weekend_df.Driver:
#     driver_gain_loss_overtake[driver] = {}
#     driver_gain_loss_overtake[driver]['gain_loss'] = weekend_df.loc[weekend_df['Driver'] == driver][[f'positions_gained_{track_name}']].values[0][0]
#     driver_gain_loss_overtake[driver]['overtake'] = weekend_df.loc[weekend_df['Driver'] == driver][[f'overtake_bonus_{track_name}']].values[0][0]

### Driver and Constructor Prices

In [6]:
# pricing for drivers and constructors, updated each week with the latest values from the Formula 1 Fantasy Game
driver_pricing = pd.read_csv('https://docs.google.com/spreadsheets/d/14kBO9LAo4-uPrQlH6xm_Fm2OcNB15xUdnzUbIaRFjOU/export?format=csv&gid=920234107').dropna(axis=1, how='all')
constructor_pricing = pd.read_csv('https://docs.google.com/spreadsheets/d/14kBO9LAo4-uPrQlH6xm_Fm2OcNB15xUdnzUbIaRFjOU/export?format=csv&gid=1972297135').dropna(axis=1, how='all')

### Evaulate Next Race

In [7]:
# F-winning
my_constructors = ['Ferrari', 'VisaCashApp RB']
my_drivers = ['Daniel Ricciardo', 'Carlos Sainz Jr.', 'Fernando Alonso', 'Oscar Piastri', 'Nico Hulkenberg']
remaining_cost_cap = 3.5

# # F-winning-maths
# my_constructors = ['Ferrari', 'VisaCashApp RB']
# my_drivers = ['Max Verstappen', 'Daniel Ricciardo', 'Charles Leclerc', 'Alexander Albon', 'Nico Hulkenberg']
# remaining_cost_cap = 0.7

In [8]:
top_teams = main(my_drivers, my_constructors, weekend_df, track_name, driver_pricing, constructor_pricing, remaining_cost_cap)

=== Predicted Driver Scores for Australia ===
{'Max Verstappen': 45, 'Sergio Perez': 29, 'Charles Leclerc': 23, 'Carlos Sainz Jr.': 19, 'Fernando Alonso': 19, 'George Russell': 13, 'Lewis Hamilton': 10, 'Lando Norris': 7, 'Oscar Piastri': 4, 'Lance Stroll': 2, 'Alexander Albon': 0, 'Daniel Ricciardo': 4, 'Esteban Ocon': 10, 'Guanyu Zhou': 4, 'Kevin Magnussen': -3, 'Logan Sargeant': 6, 'Nico Hulkenberg': -4, 'Pierre Gasly': 4, 'Valtteri Bottas': -2, 'Yuki Tsunoda': -5}

=== Current Team ===
Constructors: ['Ferrari', 'VisaCashApp RB']
Drivers: ['Daniel Ricciardo', 'Carlos Sainz Jr.', 'Fernando Alonso', 'Oscar Piastri', 'Nico Hulkenberg']
Current Team Value: 101.5
Current Available Value: 3.5
Total Number of Team Combinations: 697680
Total Number of Team Combinations I can afford: 429457
Explored all of the valid 429457 teams.

=== TEAM AT POSITION 1 WITH SCORE 180 ===
Constructor: ('Red Bull Racing-RBPT', 'Ferrari')
Drivers: ('Sergio Perez', 'Daniel Ricciardo', 'Esteban Ocon', 'Logan Sar